In [1]:
import pandas as pd 
import numpy as np

In [2]:
raw_sessiondata = pd.read_csv("20210113-133853.csv", skiprows=6, delimiter = ';')

In [3]:
#Add a column for 'timestamps'
import time
import datetime
import ciso8601

timestamp = []
for i in range(len(raw_sessiondata.index)):
    t = raw_sessiondata['PC-TIME'].iloc[i]
    ts= ciso8601.parse_datetime(t)
    if i == 0:
        t0= time.mktime(ts.timetuple())+ts.microsecond/1000000
        timestamp.append(np.around(t0-t0,3))
    else:
        tnow = time.mktime(ts.timetuple())+ts.microsecond/1000000
        timestamp.append(np.around(tnow-t0,3))

raw_sessiondata['timestamps'] = timestamp

In [4]:
df_group_info = raw_sessiondata.groupby('+INFO')
df_group_type = raw_sessiondata.groupby('TYPE')

## Audio Timestamp

In [5]:
# Find Audio timestamps ('BNC1High'), process so no duplicates
df_bnc1high = df_group_info.get_group('BNC1High')
a = df_bnc1high['timestamps'].diff().fillna(df_bnc1high['timestamps']) > 10
timestamps_bnc1high = df_bnc1high.loc[a].timestamps.to_numpy()

## Creating DataFrame: 'session.audio'

In [6]:
# intialise data of lists. 
data = {'timestamps':timestamps_bnc1high} 
# Create DataFrame 
df_audio = pd.DataFrame(data)

In [7]:
df_audio

,timestamps
0,21.372
1,62.148
2,82.933
3,113.702
4,134.487
...,...
70,2174.396
71,2195.168
72,2225.922
73,2266.626


In [8]:
df_audio.to_hdf('session.audio.h5', key = 'dfaudio')